# Pipelines to evaluate fine-mapping benchmarks

This document contains several workflows to extract results from fine-mapping benchmarks and make plots for (mostly) SuSiE manuscript.

In [ ]:
[global]
parameter: cwd = path('~/GIT/lab-dsc/dsc-finemap')
parameter: version = '20190610'
# set to 0 to use estimated prior, or another number to use specified prior
# as configured in the DSC
parameter: susie_prior = [0.1, 0.0]
# null weight options to evaluate
parameter: null_weight = [0.0]
parameter: output_dir = path(f'{cwd:a}/output/susie_paper')

def fmtP(x):
    return str(x).replace(".", "p")

## Query results

In [ ]:
[pip_1, cali_1]
depends: R_library('tibble')
output: f'{output_dir}/result_{version}.rds'
R: expand = '${ }', workdir = cwd
    library(tibble)
    dap_out = dscrutils::dscquery(${output_dir:r},
                        targets = c("small_data.dataset", "lm_pve02.meta", "lm_pve02.n_signal", 
                                  "score_dap.total", "score_dap.valid", "score_dap.size", "score_dap.avgr2", 
                                  "score_dap.top", "score_dap.overlap", "score_dap.signal_pip", "score_dap.pip"))
    susie_out = dscrutils::dscquery(${output_dir:r},
                        targets = c("small_data.dataset", "lm_pve02.meta", "lm_pve02.n_signal", "susie.prior_var", "susie.null_weight", 
                                  "score_susie.total", "score_susie.valid", "score_susie.size", "score_susie.purity", "score_susie.top", 
                                  "score_susie.objective", "score_susie.converged", "score_susie.overlap", "score_susie.signal_pip", "score_susie.pip"))
    caviar_out = dscrutils::dscquery(${output_dir:r},
                        targets = c("small_data.dataset", "lm_pve02.meta", "lm_pve02.n_signal", "caviar_in_sample.args", 
                                  "score_caviar.total", "score_caviar.valid", "score_caviar.size", " score_caviar.signal_pip", "score_caviar.pip"))
    finemap_out = dscrutils::dscquery(${output_dir:r},
                        targets = c("small_data.dataset", "lm_pve02.meta", "lm_pve02.n_signal", "finemap_in_sample.args", 
                                  "score_finemap.total", "score_finemap.valid", "score_finemap.size", "score_finemap.signal_pip", "score_finemap.pip"))
    rename_cols = function(dat) {
      for (item in names(dat)) {
          tmp = strsplit(colnames(dat[[item]]), "[.]")
          colnames(dat[[item]]) = unlist(lapply(1:length(tmp), function(i) ifelse(length(tmp[[i]])>1, tmp[[i]][2], tmp[[i]][1])))
      }
      return(dat)
    }
    # remove module names from column names; this is Okay because every column field here are unique
    res = rename_cols(list(dap=as_tibble(dap_out), susie=as_tibble(susie_out), caviar=as_tibble(caviar_out), finemap=as_tibble(finemap_out)))
    saveRDS(res, ${_output:r})

## PIP extraction

In [ ]:
[pip_2, cali_2]
depends: R_library('tibble'), R_library('dplyr')
input: for_each = ['susie_prior', 'null_weight']
output: f'{_input:n}_prior_{fmtP(_susie_prior)}_null_{fmtP(_null_weight)}.rds'
R: stdout = f'{_output:n}.log', expand = '${ }', workdir = cwd
    dat = readRDS(${_input:r})
    dap_out = dat$dap
    caviar_out = dat$caviar
    susie_out = dat$susie
    finemap_out = dat$finemap
    # favorit susie flavor
    susie_out = susie_out[which(susie_out$prior_var == ${_susie_prior} & susie_out$null_weight == ${_null_weight}), ]
    # and for the rest of the PIP analysis we pool across all data-set
    # but we evaluate for each number of signals settings
    # PVE is fixed to 0.2 here using `lm_pve02` module
    data_sets = unique(susie_out$dataset)
    n_signals = unique(susie_out$n_signal)
    result = list()
    for (s in n_signals) {
        result[[as.character(s)]] = NULL
        if (s > 3) {
            has_caviar = FALSE
        } else {
            has_caviar = TRUE
        }
        print(paste('==============', s, '=============='))
        for (d in data_sets) {
            # should all be one row
            susie_row = susie_out[which(susie_out$n_signal == s & susie_out$dataset == d), ]
            dap_row = dap_out[which(dap_out$n_signal == s & dap_out$dataset == d), ]
            if (nrow(susie_row) != 1 || nrow(dap_row) != 1) stop("Rows are not unique for susie & dap")
            if (has_caviar) {
              caviar_row = caviar_out[which(caviar_out$n_signal == s & caviar_out$dataset == d & caviar_out$args == paste('-g 0.001 -c', s)), ]
              finemap_row = finemap_out[which(finemap_out$n_signal == s & finemap_out$dataset == d & finemap_out$args == paste('--n-causal-max', s)), ]
              if (nrow(caviar_row) != 1 || nrow(finemap_row) != 1) stop("Rows are not unique for caviar & finemap")
            }
            # a slightly awkward yet convenient syntax thanks to tibble format
            truth = susie_row$meta[[1]]$true_coef
            for (r in 1:1) {
              susie_pip = susie_row$pip[[1]][,r]
              dap_pip = dap_row$pip[[1]][,r]
              if (has_caviar) {
                  caviar_pip = caviar_row$pip[[1]][,r]
                  finemap_pip = finemap_row$pip[[1]][,r]
                  pip = cbind(susie_pip, dap_pip, caviar_pip, finemap_pip, truth[,r] != 0)
              } else {
                  pip = cbind(susie_pip, dap_pip, truth[,r] != 0)
              }
              # remove all zero PIP / table
              pip = pip[rowSums(pip) > 0, ]
              ## BEGIN debug
              outlier = pip[which(pip[,1] < 0.1 & pip[,2]>0.6), ,drop=F]
              if (nrow(outlier)>0 && s <= 2) {
                print("DAP outlier")
                print(c(r,d))
              }
              if (has_caviar && s == 1) {
                conflict = pip[which(pip[,1] < 0.95 & pip[,3] > 0.95), ,drop=F]
                if (nrow(conflict) > 0) {
                    print("CAVIAR-susie conflict")
                    print(d)
                    print(pip[conflict,])
                    print("CAVIAR")
                    print(which(caviar_pip>0.95))
                    print("SuSiE")
                    print(which(susie_pip>0.95))
                }
                outlier = pip[which(pip[,1] > 0.1 & pip[,3] < 0.1), ,drop=F]
                if (nrow(outlier)>0 && s <= 2) {
                    print("SuSiE outlier")
                    print(c(r,d))
                }
              }
              ## END debug
              if (is.null(result[[as.character(s)]])) {
                  result[[as.character(s)]] = pip
              } else {
                  result[[as.character(s)]] = rbind(result[[as.character(s)]], pip)
              }
           }
        }
        result[[as.character(s)]] = data.frame(result[[as.character(s)]])
        if (has_caviar) {
            colnames(result[[as.character(s)]]) = c('susie', 'dap', 'caviar', 'finemap', 'is_signal')
        } else {
            colnames(result[[as.character(s)]]) = c('susie', 'dap', 'is_signal')
        }
    }
    saveRDS(result, ${_output:r})

## PIP direct comparison

In [ ]:
[pip_3]
comparisons = ['susie_vs_dap', 'susie_vs_caviar', 'susie_vs_finemap', 'dap_vs_caviar', 'dap_vs_finemap', 'caviar_vs_finemap']
input: group_by = 1, concurrent = True
output: paths([f'{_input:n}.{x}.png' for x in comparisons])
R: expand = '${ }'
    result = readRDS(${_input:r})
    merge_img = function(prefix, n) {
        files = paste0(prefix, '_', seq(1:n), '.png')
        cmd = paste('convert +append', paste(files, collapse=" "), paste0(prefix, '.png'))
        system(cmd)
        system(paste('rm -f', paste(files, collapse=" ")))
        files = paste0(prefix, '_', seq(1:n), '.md')
        cmd = paste('cat', paste(files, collapse=" "), '>', paste0(prefix, '.md'))
        system(cmd)
        system(paste('rm -f', paste(files, collapse=" ")))
    }

    merge_lists = function(lists) {
        lists = do.call(rbind,lapply(cbind(lists), unlist))
        names = colnames(lists)
        lists = lapply(1:ncol(lists), function(i) lists[,i])
        names(lists) = names
        return(lists)
    }

    plot_pip = function(x, n_causal, s, output_prefix, xname, yname, xlab, ylab, pip_cutoff = -1, pip_diff_categories = c(0.1, 0.15, 0.2)) {
      x = x[x[[xname]] > pip_cutoff & x[[yname]] > pip_cutoff,]
      if (pip_cutoff >=0) {
          xlab = paste0(xlab, ">", pip_cutoff)
          ylab = paste0(ylab, ">", pip_cutoff)
      }
      colors = sapply(1:length(x$is_signal), function(i) ifelse(x$is_signal[i],'#800000','#002b36'))
      # compute difference in pip and proportions
      pip_diff = abs(x[[xname]] - x[[yname]])
      pip_diff_cnts = sapply(pip_diff_categories, function(x) length(which(pip_diff >= x)))
      pip_diff_text = vector()
      for (i in 1:length(pip_diff_categories)) {
          tmp = paste0('- ', pip_diff_cnts[i], '/', length(pip_diff), ' (', formatC(pip_diff_cnts[i] / length(pip_diff) * 100, format = "e", digits = 2), '%) differ by ', pip_diff_categories[i])
          pip_diff_text = c(pip_diff_text, tmp)
      }
      pip_diff_text = paste(pip_diff_text, collapse = '\n')
      corr_text = paste('- correlation', round(cor(x)[1,2],2))
      header = paste('#', xname, 'vs', yname, s, n_causal, 'causal\n')
      pdf(paste0(output_prefix, '_', n_causal, '.pdf'), width=5, height=5, pointsize=14)
      plot(x[[xname]], x[[yname]], xlab = xlab, ylab = ylab,
         main = paste0(n_causal, s , ifelse(n_causal>1, ' effect variables', ' effect variable')),
         col = colors, pch = 20, cex = 1.4, bty='l')
      abline(0,1,col=2)
      #abline(h=0.95, col='gray')
      #abline(v=0.95, col='gray')
      dev.off()
      system(paste0("convert -flatten -density 120 ", paste0(output_prefix, '_', n_causal, '.pdf'), " ", paste0(output_prefix, '_', n_causal, '.png')))
      write(paste(header, corr_text, pip_diff_text, '\n', sep = '\n'), paste0(output_prefix, '_', n_causal, '.md'))
    }
    result_1 = result[[1]]
    result_2 = result[[2]]
    result_3 = result[[3]]
    result_5 = do.call(rbind, lapply(3:5, function(i) result[[i]][,c('susie','dap', 'is_signal')]))
    # susie vs dap
    plot_pip(result_1, 1, '', ${_output[0]:nr}, 'susie', 'dap', 'PIP SuSiE', 'PIP DAP-G')
    plot_pip(result_2, 2, '', ${_output[0]:nr}, 'susie', 'dap', 'PIP SuSiE', 'PIP DAP-G')
    plot_pip(result_5, 3, ' ~ 5', ${_output[0]:nr}, 'susie', 'dap', 'PIP SuSiE', 'PIP DAP-G')
    merge_img(${_output[0]:nr}, 3)
    # susie vs caviar
    plot_pip(result_1, 1, '', ${_output[1]:nr}, 'susie', 'caviar', 'PIP SuSiE', 'PIP CAVIAR')
    plot_pip(result_2, 2, '', ${_output[1]:nr}, 'susie', 'caviar', 'PIP SuSiE', 'PIP CAVIAR')
    plot_pip(result_3, 3, '', ${_output[1]:nr}, 'susie', 'caviar', 'PIP SuSiE', 'PIP CAVIAR')
    merge_img(${_output[1]:nr}, 3)
    # susie vs finemap
    plot_pip(result_1, 1, '', ${_output[2]:nr}, 'susie', 'finemap', 'PIP SuSiE', 'PIP FINEMAP')
    plot_pip(result_2, 2, '', ${_output[2]:nr}, 'susie', 'finemap', 'PIP SuSiE', 'PIP FINEMAP')
    plot_pip(result_3, 3, '', ${_output[2]:nr}, 'susie', 'finemap', 'PIP SuSiE', 'PIP FINEMAP')
    merge_img(${_output[2]:nr}, 3)
    # dap vs caviar
    plot_pip(result_1, 1, '', ${_output[3]:nr}, 'dap', 'caviar', 'PIP DAP-G', 'PIP CAVIAR')
    plot_pip(result_2, 2, '', ${_output[3]:nr}, 'dap', 'caviar', 'PIP DAP-G', 'PIP CAVIAR')
    plot_pip(result_3, 3, '', ${_output[3]:nr}, 'dap', 'caviar', 'PIP DAP-G', 'PIP CAVIAR')
    merge_img(${_output[3]:nr}, 3)
    # dap vs finemap
    plot_pip(result_1, 1, '', ${_output[4]:nr}, 'dap', 'finemap', 'PIP DAP-G', 'PIP FINEMAP')
    plot_pip(result_2, 2, '', ${_output[4]:nr}, 'dap', 'finemap', 'PIP DAP-G', 'PIP FINEMAP')
    plot_pip(result_3, 3, '', ${_output[4]:nr}, 'dap', 'finemap', 'PIP DAP-G', 'PIP FINEMAP')
    merge_img(${_output[4]:nr}, 3)
    # caviar vs finemap
    plot_pip(result_1, 1, '', ${_output[5]:nr}, 'caviar', 'finemap', 'PIP CAVIAR', 'PIP FINEMAP')
    plot_pip(result_2, 2, '', ${_output[5]:nr}, 'caviar', 'finemap', 'PIP CAVIAR', 'PIP FINEMAP')
    plot_pip(result_3, 3, '', ${_output[5]:nr}, 'caviar', 'finemap', 'PIP CAVIAR', 'PIP FINEMAP')
    merge_img(${_output[5]:nr}, 3)